In [1]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import requests
from census import Census

# Census API Key
from config import api_key

c = Census(api_key, year=2010)


In [2]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), 
                         {'for': 'county:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", 
                                      "county": "County"})

census_pd["Name"] = census_pd["Name"].str.split('County', expand=True)
census_pd["Name"] = census_pd["Name"].str.split('Municipio', expand=True)
census_pd['Name'] = census_pd['Name'].str.upper() 
  
census_pd.head()

,Median Age,Population,Poverty Count,Household Income,Per Capita Income,Name,County,state
0,35.8,44631.0,21672.0,16564.0,8949.0,MANATÍ,091,72
1,29.3,250699.0,53738.0,43844.0,18041.0,MERCED,047,06
2,45.4,9605.0,1709.0,34588.0,20536.0,MODOC,049,06
3,36.5,13905.0,1664.0,55087.0,27321.0,MONO,051,06
4,32.8,407435.0,54276.0,59271.0,25776.0,MONTEREY,053,06


In [3]:
# File to Load (Remember to Change These)
file_to_load = "Data/All_Places_of_Worship.csv"

# Read Purchasing File and store into Pandas data frame
worship = pd.read_csv(file_to_load)
worship.head()

,OBJECTID,ID,NAME,TELEPHONE,ADDRESS,ADDRESS2,CITY,STATE,ZIP,ZIPP4,...,PHONELOC,QC_QA,STATE_ID,SUBTYPE,DENOM,MEMBERS,ATTENDANCE,LOC_TYPE,PROT,CATH
0,1001,10438385.0,FREMONT PRESBYTERIAN CHURCH,916-452-7132,5770 CARLSON DRIVE,NOT AVAILABLE,SACRAMENTO,CA,95819,2553.0,...,NaN,TGS,NOT AVAILABLE,CHRISTIAN,PCUSA,1292,800,NOT AVAILABLE,Y,N
1,1002,10438397.0,LAFAYETTE - ORINDA PRESBYTERIAN CHURCH,925-283-8722,49 KNOX DRIVE,NOT AVAILABLE,LAFAYETTE,CA,94549,3322.0,...,NaN,TGS,NOT AVAILABLE,CHRISTIAN,PCUSA,1971,775,NOT AVAILABLE,Y,N
2,1003,10438425.0,WALNUT CREEK PRESBYTERIAN CHURCH,925-935-1574,1801 LACASSIE AVENUE,NOT AVAILABLE,WALNUT CREEK,CA,94596,4016.0,...,NaN,TGS,NOT AVAILABLE,CHRISTIAN,PCUSA,1290,914,NOT AVAILABLE,Y,N
3,1004,10431831.0,MUHAJEREEN MOSQUE,510-786-0313,185 FOLSOM AVENUE,NOT AVAILABLE,HAYWARD,CA,94544,6504.0,...,NaN,TGS,NOT AVAILABLE,MUSLIM,NOT AVAILABLE,NOT AVAILABLE,0,NOT AVAILABLE,N,N
4,1005,10438443.0,THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINT...,916-422-6919,7401 24TH STREET,NOT AVAILABLE,SACRAMENTO,CA,95822,5319.0,...,NaN,TGS,NOT AVAILABLE,CHRISTIAN,LDS,NOT AVAILABLE,0,NOT AVAILABLE,Y,N


In [6]:
worship_clean = worship.loc[:,["OBJECTID","GEOLINKID", "NAME", "ADDRESS", 
                               "CITY", "STATE", "COUNTY", "SUBTYPE", 
                               "DENOM","PROT", "CATH", "X", "Y", "CONTDATE"]]
worship_clean=worship_clean.rename(columns={"COUNTY": "Name"})
worship_clean.head(10)

,OBJECTID,GEOLINKID,NAME,ADDRESS,CITY,STATE,Name,SUBTYPE,DENOM,PROT,CATH,X,Y,CONTDATE
0,1001,24458797.0,FREMONT PRESBYTERIAN CHURCH,5770 CARLSON DRIVE,SACRAMENTO,CA,SACRAMENTO,CHRISTIAN,PCUSA,Y,N,-121.429885,38.567942,2007-10-10T00:00:00.000Z
1,1002,23660044.0,LAFAYETTE - ORINDA PRESBYTERIAN CHURCH,49 KNOX DRIVE,LAFAYETTE,CA,CONTRA COSTA,CHRISTIAN,PCUSA,Y,N,-122.158534,37.887245,2007-10-10T00:00:00.000Z
2,1003,24597871.0,WALNUT CREEK PRESBYTERIAN CHURCH,1801 LACASSIE AVENUE,WALNUT CREEK,CA,CONTRA COSTA,CHRISTIAN,PCUSA,Y,N,-122.067842,37.902615,2007-10-10T00:00:00.000Z
3,1004,24523694.0,MUHAJEREEN MOSQUE,185 FOLSOM AVENUE,HAYWARD,CA,ALAMEDA,MUSLIM,NOT AVAILABLE,N,N,-122.058762,37.628087,2008-10-31T00:00:00.000Z
4,1005,24466544.0,THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINT...,7401 24TH STREET,SACRAMENTO,CA,SACRAMENTO,CHRISTIAN,LDS,Y,N,-121.479597,38.489979,2007-10-15T00:00:00.000Z
5,1006,23744058.0,MUHAJEREEN MOSQUE,27949 LEIDIG COURT,HAYWARD,CA,ALAMEDA,MUSLIM,NOT AVAILABLE,N,N,-122.064728,37.634874,2008-11-05T00:00:00.000Z
6,1007,23735858.0,SAINT FRANCES CABRINI CATHOLIC CHURCH,15333 WOODARD ROAD,SAN JOSE,CA,SANTA CLARA,CHRISTIAN,NOT AVAILABLE,N,Y,-121.937232,37.261029,2008-11-12T00:00:00.000Z
7,1008,23735858.0,KNANAYA CATHOLIC COMMUNITY OF NORTHERN CALIFORNIA,15333 WOODARD ROAD,SAN JOSE,CA,SANTA CLARA,CHRISTIAN,NOT AVAILABLE,N,Y,-121.937235,37.261031,2007-08-31T00:00:00.000Z
8,1009,37692802.0,THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINT...,550 NORTH DANEBO AVENUE,EUGENE,OR,LANE,CHRISTIAN,LDS,Y,N,-123.178777,44.066847,2007-10-16T00:00:00.000Z
9,1010,28485032.0,THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINT...,3551 DECOTO ROAD,FREMONT,CA,ALAMEDA,CHRISTIAN,LDS,Y,N,-122.029872,37.574521,2007-10-12T00:00:00.000Z


In [7]:
Final = pd.merge(worship_clean, census_pd,
                        how="left", on=["Name", "Name"])
Final.head()


,OBJECTID,GEOLINKID,NAME,ADDRESS,CITY,STATE,Name,SUBTYPE,DENOM,PROT,...,X,Y,CONTDATE,Median Age,Population,Poverty Count,Household Income,Per Capita Income,County,state
0,1001,24458797.0,FREMONT PRESBYTERIAN CHURCH,5770 CARLSON DRIVE,SACRAMENTO,CA,SACRAMENTO,CHRISTIAN,PCUSA,Y,...,-121.429885,38.567942,2007-10-10T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,23660044.0,LAFAYETTE - ORINDA PRESBYTERIAN CHURCH,49 KNOX DRIVE,LAFAYETTE,CA,CONTRA COSTA,CHRISTIAN,PCUSA,Y,...,-122.158534,37.887245,2007-10-10T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,24597871.0,WALNUT CREEK PRESBYTERIAN CHURCH,1801 LACASSIE AVENUE,WALNUT CREEK,CA,CONTRA COSTA,CHRISTIAN,PCUSA,Y,...,-122.067842,37.902615,2007-10-10T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,24523694.0,MUHAJEREEN MOSQUE,185 FOLSOM AVENUE,HAYWARD,CA,ALAMEDA,MUSLIM,NOT AVAILABLE,N,...,-122.058762,37.628087,2008-10-31T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,24466544.0,THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINT...,7401 24TH STREET,SACRAMENTO,CA,SACRAMENTO,CHRISTIAN,LDS,Y,...,-121.479597,38.489979,2007-10-15T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
Final.to_csv(r'Worship_Census_Merge.csv', index = False)
